In [2]:
from index import create_index, create_llm


index = create_index()
llm = create_llm()

index, llm

(<llama_index.core.indices.vector_store.base.VectorStoreIndex at 0x21e3198e0f0>,
 Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x0000021E2F921D30>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x0000021E1FCC1DA0>, completion_to_prompt=<function default_completion_to_prompt at 0x0000021E1FD9BF60>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://localhost:11434', model='qwen2:7b', temperature=0.75, context_window=3900, request_timeout=30.0, prompt_key='prompt', json_mode=False, additional_kwargs={}, is_function_calling_model=True, keep_alive=None))

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-7B-Instruct")

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

c:\Users\mikoo\miniconda3\envs\primo\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mikoo\.cache\huggingface\hub\models--Qwen--Qwen2-7B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

#### as query engine

In [3]:
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("Kto prowadzi rejestr obywateli Ukrainy?")
response

Response(response='Minister właściwy do spraw informatyzacji prowadzi rejestr obywateli Ukrainy.', source_nodes=[NodeWithScore(node=TextNode(id_='4371c3d3-f643-41c9-a468-cd9e9b994a18', embedding=None, metadata={'page_label': '5', 'file_name': 'specustawa_12032022.pdf', 'file_path': 'C:\\Users\\mikoo\\Downloads\\specustawa_12032022.pdf', 'file_type': 'application/pdf', 'file_size': 832598, 'creation_date': '2025-03-13', 'last_modified_date': '2025-03-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='383d67d6-6398-470b-bff7-13d29d41719a', node_type='4', metadata={'page_label': '5', 'file_name': 'specustawa_12032022.pdf', 'file_path': 'C:\\Users\\mikoo\\Downloads\\specustawa_12032022.pdf', 'file_type': 'ap

In [4]:
print(f"Typ:  {type(response)}")
print(f"Tekst:  {response.response}")
print(f"Źródło:  {response.get_formatted_sources()}")

Typ:  <class 'llama_index.core.base.response.schema.Response'>
Tekst:  Minister właściwy do spraw informatyzacji prowadzi rejestr obywateli Ukrainy.
Źródło:  > Source (Doc id: 4371c3d3-f643-41c9-a468-cd9e9b994a18): Art. 5. 1. Organ gminy może zapewnić obywatelowi Ukrainy, którego pobyt na terytorium Rzeczypospo...


In [5]:
query_engine = index.as_query_engine(llm=llm, streaming=True)
streaming_response = query_engine.query("Kto prowadzi rejestr obywateli Ukrainy?")
streaming_response.print_response_stream()

Rejestr obywateli Ukrainy prowadzi Minister właściwy do spraw informatyzacji.

In [6]:
print(streaming_response.response_gen)
print(streaming_response.get_formatted_sources())
print(streaming_response.get_response())
print(streaming_response.print_response_stream())

<generator object stream_chat_response_to_tokens.<locals>.gen at 0x0000021E3AD8EB50>
> Source (Node id: 4371c3d3-f643-41c9-a468-cd9e9b994a18): Art. 5. 1. Organ gminy może zapewnić obywatelowi Ukrainy, którego pobyt na terytorium Rzeczypospo...
Rejestr obywateli Ukrainy prowadzi Minister właściwy do spraw informatyzacji.
Rejestr obywateli Ukrainy prowadzi Minister właściwy do spraw informatyzacji.
None


Response modes

In [16]:
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="compact",
)
response = query_engine.query("O czym mówi ustawa?")
print(response)

Ustawa dotyczy przepisów dotyczących wykonywania prac w obszarach pokrytych wodami, w pasie drogowym oraz wykorzystywania kanałów technologicznych. Zdefiniowane są warunki dozwolone dla eksploatacji instalacji i wymagania dotyczące zgłoszenia do właściwego organu ochrony środowiska. Ustawa także omagaćcy problem obciążenia nieruchomości usługą wykonawców prac w pasie drogowym.


#### 

In [17]:
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="refine",
)
response = query_engine.query("O czym mówi ustawa?")
print(response)

Ustawa odnosi się do różnych aspektów praw dotyczących użytkowania gruntów pokrytych wodami i wykonywania prac na nich. Ustawę to regulują przepisy dotyczące zawarcia umowy, wykonywania prac w obrębie pasa drogowego bez wymagania sporządzania projektu organizacji ruchu czy płacenia opłat, korzystanie z kanału technologicznego na drogach publicznych oraz eksploatacji instalacji. Ustawa również omaga to co do zgłoszenia w odniesieniu do ochrony środowiska, a także przepisy dotyczące obciążyności nieruchomości usługą polegającą na wykonywaniu określonych prac przez operatora telekomunikacyjnego.


In [19]:
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
)
response = query_engine.query("O czym mówi ustawa?")
print(response)

Ustawa mówi o różnych aspektach wykonywania prac na gruntach pokrytych wodami, np. zawarciu umowy użytkowania takich gruntów, wykonywaniu prac w pasie drogowym bez wymagania sporządzania projektu organizacji ruchu lub pobierania opłat, korzystaniu z kanałów technologicznych na drogach publicznych oraz przystąpieniu do eksploatacji instalacji. Tekst również omawia odpowiednie zgłoszenia dla operatora telekomunikacyjnego oraz obciążyność nieruchomości służbą związującą wykonywanie określonych prac.


In [20]:
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="accumulate",
)
response = query_engine.query("O czym mówi ustawa?")
print(response)

Response 1: Ustawa mówiona w kontekście odnosi się do Prawa wodnego z dnia 20 lipca 2017 roku oraz do ustawy o drogach publicznych z dnia 21 marca 1985 roku. W niej są omawiane kwestie związane z zawarciem umowy dotyczącej użytkowania gruntów pokrytych wodami, wykonywaniem prac w pasie drogowym bez konieczności sporządzania projektu organizacji ruchu i opłat za taką akcję, wykorzystywaniem kanałów technologicznych na drogach publicznych oraz prawem eksploatacji instalacji. Ustawa również omaga się z przepisami dotyczącymi ochrony środowiska, podkreślając wymagania związanych z zgłoszeniem operacji prowadzonej przez operatora telekomunikacyjnego.


In [21]:
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="simple_summarize",
)
response = query_engine.query("O czym mówi ustawa?")
print(response)

Ustawa porusza tematy związane z zawarciem umowy w sprawie użytkowania gruntów pokrytych wodami, wykonywaniem prac w obrębie pasa drogowego, wykorzystaniem kanału technologicznego na drogach publicznych, przystąpieniem do eksploatacji instalacji oraz obciążeniem nieruchomości usługą.


Nie widze aby sposob wyswietlania mial wieksze znaczenie niz losowosc odpowiedzi

In [38]:
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="simple_summarize",
)
response = query_engine.query("O czym mówi ustawa?")
print(response)

Ustawa mowa o zawarciu umowy w sprawie użytkowania gruntów pokrytych wodami, regulacji wykonywania prac w obrębie pasa drogowego drogi publicznej bez wymagania sporządzenia projektu organizacji ruchu i bez pobierania opłaty, wykorzystaniu kanału technologicznego do wykonywania prac na drogach publicznych, przystąpieniu do eksploatacji instalacji przed doręceniem zgłoszenia w organ ochrony środowiska zgodnie z danymi przepisami prawa, oraz prawie operatora telekomunikacyjnego wykonywającego takich prac.


In [41]:
print("\nŹródła:")
for node in response.source_nodes:
    print(f"  * Dokument: {node.node.ref_doc_id}\n")
    print(f"    Fragment tekstu: {node.node.get_content()}\n")
    print(f"    Podobieństwo: {node.score}\n")
    print("-" * 40)


Źródła:
  * Dokument: 3c631f9f-4ef7-42b0-8ff0-0e4d2761e31a

    Fragment tekstu: ©Kancelaria Sejmu  s. 40/45 
2022-03-14 
c) zawarcia umowy w sprawie użytkowania gruntów pokrytych wodami, o  której mowa w art. 261 ust. 1 
pkt 8 ustawy z dnia 20 lipca 2017 r. – Prawo wodne (Dz.  U. z 2021 r. poz. 2233 i 2368 oraz z  2022 r. 
poz. 88 i 258); 
2) w przypadku, gdy wykonywanie tych prac ma nastąpić w  obrębie pasa drogowego drogi publicznej – może 
nastąpić po poinformowaniu właściwego zarządcy drogi o  planowanym zakresie zajęcia pasa drogowego, 
przy czym nie jest wymagane sporządzenie projektu organizacji ruchu, o którym mowa w przepisach wyko-
nawczych wydanych na podstawie art. 10 ust. 12 ustawy z dnia 20 czerwca 1997 r. – Prawo o ruchu drogo-
wym (Dz. U. z 2021 r. poz. 450, z późn. zm.12)), oraz nie pobiera się opłaty, o  której mowa w art. 40 ust. 3 
ustawy z dnia 21 marca 1985 r. o drogach publicznych; 
3) w przypadku, gdy do wykonywania tych prac możliwe jest wykorzystanie kanału 

In [49]:
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
)
response = query_engine.query("Czego dotyczy ustawa o pomocy obywatelom Ukrainy?")
print(response)

Ustawa o pomocy obywatelom Ukrainy dotyczy szczegółów regulacji dotyczących pobytu obywateli Ukrainy, którzy przybyli na terytorium Rzeczypospolitej Polskiej bezpośrednio z terytorium Ukrainy w związku z działaniami wojennymi prowadzonymi na terytorium tego państwa. Ustawa obejmuje również pomoc zapewnianą przez odpowiednich podmiotów, utworzenie Funduszu Pomocy oraz niektóre uprawnienia obywateli Ukrainy i Polski związanych z wjeżdżaniem i pobytu na terytorium Rzeczypospolitej Polskiej.


In [43]:
print("\nŹródła:")
for node in response.source_nodes:
    print(f"  * Dokument: {node.node.ref_doc_id}\n")
    print(f"    Fragment tekstu: {node.node.get_content()}\n")
    print(f"    Podobieństwo: {node.score}\n")
    print("-" * 40)


Źródła:
  * Dokument: f75563de-ebb4-4f5c-82a0-106931bd1a8a

    Fragment tekstu: ©Kancelaria Sejmu  s. 1/45 
2022-03-14 
 
Dz.U. 2022 poz. 583 
 
USTAWA  
z dnia 12 marca 2022 r. 
o pomocy obywatelom Ukrainy w związku z konfliktem zbrojnym na terytorium tego państwa1) 
Art. 1. 1. Ustawa określa szczególne zasady zalegalizowania pobytu obywateli Ukrainy, którzy przybyli na terytorium 
Rzeczypospolitej Polskiej bezpośrednio z terytorium Ukrainy w związku z działaniami wojennymi prowadzonymi na terytorium 
tego państwa, oraz obywateli Ukrainy posiadających Kartę Polaka, którzy wraz z najbliższą rodziną z powodu tych działań 
wojennych przybyli na terytorium Rzeczypospolitej Polskiej. 
2. Ilekroć w ustawie jest mowa o obywatelu Ukrainy, rozumie się przez to także nieposiadającego obywatelstwa ukraińskiego 
małżonka obywatela Ukrainy, o ile przybył on na terytorium Rzeczypospolitej Polskiej bezpośrednio z  terytorium Ukrainy 
w związku z działaniami wojennymi prowadzonymi na terytorium teg

In [48]:
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree",
)
response = query_engine.query("Od kogo nie można pobierać odcisków palców?")
print(response)

ValueError: Unknown mode: tree

In [47]:
print("\nŹródła:")
for node in response.source_nodes:
    print(f"  * Dokument: {node.node.ref_doc_id}\n")
    print(f"    Fragment tekstu: {node.node.get_content()}\n")
    print(f"    Podobieństwo: {node.score}\n")
    print("-" * 40)


Źródła:
  * Dokument: cc750565-a988-4da4-bf79-e9df97245345

    Fragment tekstu: 3 pkt 14 ustawy z dnia 17 lutego 2005 r. o informatyzacji 
działalności podmiotów realizujących zadania publiczne. 
6. Do danych, o których mowa w ust. 4, stosuje się sposób transliteracji określony w art. 14 ust. 2 ustawy z dnia 25 listo-
pada 2004 r. o zawodzie tłumacza przysięgłego (Dz. U. z 2019 r. poz. 1326). 
7. Do wniosku, o  którym mowa w  ust. 1, załącza się fotografię, spełniającą wymagania określone w  art. 29 ustawy 
z dnia 6 sierpnia 2010 r. o dowodach osobistych (Dz. U. z 2021 r. poz. 816, 1000 i 1978). 
8. Podczas składania wniosku, o którym mowa w ust. 1, pobiera się od osoby, której dotyczy wniosek, odciski palców. 
9. Odcisków palców nie pobiera się od osoby: 
1) która nie ukończyła 12. roku życia; 
2) od której chwilowo fizycznie nie jest możliwe pobranie odcisków któregokolwie k z palców; 
3) od której pobranie odcisków palców jest fizycznie niemożliwe. 
10. Odciski palców pobiera się 

In [13]:
# Wyświetlanie wszystkich elementów obiektu response
for attr_name in dir(response):
    if not attr_name.startswith('__'):
        attr_value = getattr(response, attr_name)
        print(f"{attr_name}: {attr_value}\n")


get_formatted_sources: <bound method Response.get_formatted_sources of Response(response='Ustawa o pomocy obywatelom Ukrainy dotyczy szczegółów dotyczących uzależnienia pobytu obywateli Ukrainy, którzy przybyli na terytorium Rzeczypospolitej Polskiej bezpośrednio z terytorium Ukrainy w związku z działaniami wojennymi prowadzonymi na terytorium tego państwa. Ustawa obejmuje także obywateli Ukrainy posiadających Kartę Polaka, którzy wraz z najbliższą rodziną z powodu tych działań wojskowych przybyli na terytorium Rzeczypospolitej Polskiej. Kwestie obejmowane w ustawie to szczegóły dotyczące powierzenia pracy obywatelom Ukrainy, pomoc zapewnianą przez wojewodów i jednostki samorządu terytorialnego dla obywateli Ukrainy, utworzenie Funduszu Pomocy do finansowania lub dofinansowania realizacji zadań na rzecz pomocy obywatelom Ukrainy, niektóre uprawnienia obywateli Ukrainy, szczególne zasady przedłużania okresów legalnego pobytu obywateli Ukrainy oraz wydanych im przez organy polskie dokume

Ustawa o pomocy obywatelom Ukrainy dotyczy szczegółów dotyczących uzależnienia pobytu obywateli Ukrainy, którzy przybyli na terytorium Rzeczypospolitej Polskiej bezpośrednio z terytorium Ukrainy w związku z działaniami wojennymi prowadzonymi na terytorium tego państwa. Ustawa obejmuje także obywateli Ukrainy posiadających Kartę Polaka, którzy wraz z najbliższą rodziną z powodu tych działań wojskowych przybyli na terytorium Rzeczypospolitej Polskiej. Kwestie obejmowane w ustawie to szczegóły dotyczące powierzenia pracy obywatelom Ukrainy, pomoc zapewnianą przez wojewodów i jednostki samorządu terytorialnego dla obywateli Ukrainy, utworzenie Funduszu Pomocy do finansowania lub dofinansowania realizacji zadań na rzecz pomocy obywatelom Ukrainy, niektóre uprawnienia obywateli Ukrainy, szczególne zasady przedłużania okresów legalnego pobytu obywateli Ukrainy oraz wydanych im przez organy polskie dokumentów dotyczących uprawnień w zakresie wjazdu i pobytu na terytorium Rzeczypospolitej Polsk

In [15]:
# Wyświetlanie odpowiedzi
print("Odpowiedź:")
print(response.response)

# Wyświetlanie metadanych źródeł
print("\nMetadane źródeł:")
for node in response.source_nodes:
    print(f"ID węzła: {node.node.id_}")
    print(f"Strona: {node.metadata['page_label']}")
    print(f" Nazwa pliku: {node.metadata['file_name']}")
    print(f"Ścieżka pliku: {node.metadata['file_path']}")
    print(f"Typ pliku: {node.metadata['file_type']}")
    print(f"Rozmiar pliku: {node.metadata['file_size']} bajtów")
    print(f"Data utworzenia: {node.metadata['creation_date']}")
    print(f"Data modyfikacji: {node.metadata['last_modified_date']}\n")


Odpowiedź:
Ustawa o pomocy obywatelom Ukrainy dotyczy szczegółów dotyczących uzależnienia pobytu obywateli Ukrainy, którzy przybyli na terytorium Rzeczypospolitej Polskiej bezpośrednio z terytorium Ukrainy w związku z działaniami wojennymi prowadzonymi na terytorium tego państwa. Ustawa obejmuje także obywateli Ukrainy posiadających Kartę Polaka, którzy wraz z najbliższą rodziną z powodu tych działań wojskowych przybyli na terytorium Rzeczypospolitej Polskiej. Kwestie obejmowane w ustawie to szczegóły dotyczące powierzenia pracy obywatelom Ukrainy, pomoc zapewnianą przez wojewodów i jednostki samorządu terytorialnego dla obywateli Ukrainy, utworzenie Funduszu Pomocy do finansowania lub dofinansowania realizacji zadań na rzecz pomocy obywatelom Ukrainy, niektóre uprawnienia obywateli Ukrainy, szczególne zasady przedłużania okresów legalnego pobytu obywateli Ukrainy oraz wydanych im przez organy polskie dokumentów dotyczących uprawnień w zakresie wjazdu i pobytu na terytorium Rzeczypospo